# Tf-Idf

## Загрузим данные

In [1]:
import pandas as pd
import numpy as np

In [40]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train

,target,text
0,ham,Hey hun-onbus goin 2 meet him. He wants 2go ou...
1,ham,I love your ass! Do you enjoy doggy style? :)
2,ham,"By the way, make sure u get train to worc fore..."
3,ham,U know we watchin at lido?
4,ham,I got it before the new year cos yetunde said ...
...,...,...
2781,ham,Set a place for me in your heart and not in yo...
2782,ham,I don't run away frm u... I walk slowly &amp; ...
2783,ham,What happened to our yo date?
2784,ham,Oh ok.. Wat's ur email?


## Сохраним метки классов

In [41]:
y_train = (train["target"] == "spam").astype(int)
y_test = (test["target"] == "spam").astype(int)

# Константное предсказание

Бейзлайн - предсказать всем письмам spam.

In [51]:
from sklearn.metrics import f1_score

f1_score(y_test, np.ones((len(y_test),)))

0.21810041573393027

# Tf-Idf

$$ w_{ij} = tf_{ij} \cdot \log \left( \frac{N}{df_{i}} \right), где $$

$tf_{ij}$ - это частота встречаемости слова/$n$-граммы $i$ в предложении $j$,

$df_{i}$ - это количество предложений, содержащих слово/$n$-грамму $i$

$N$ - общее количество предложений

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(lowercase=True, analyzer="char_wb", max_features=50_000, ngram_range=(1, 5))
vectorizer.fit(train['text'])
encode_func = lambda texts: vectorizer.transform(texts)

In [73]:
(encode_func(["Hi, brooooo"]) != 0).sum()

32

# Обучаем модель

In [74]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg = logreg.fit(encode_func(train['text']), y_train)

y_pred = logreg.predict_proba(encode_func(train['text']))[:,1]

 ## Подбираем параметры

In [75]:
opt_threshold = 0
opt_f1 = f1_score(y_train, y_pred > 0)
for threshold in np.linspace(0, 1, 1000):
    f1 = f1_score(y_train, y_pred > threshold)
    if f1 > opt_f1:
        opt_f1 = f1
        opt_threshold = threshold

y_pred = logreg.predict_proba(encode_func(test['text']))[:,1]
y_pred = y_pred > opt_threshold

f1_score(y_test, y_pred)

0.9518248175182482